In [ ]:
import yfinance as yf
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

DB_FILE = '/content/drive/MyDrive/stocks/super_trend.db'

def create_connection(db_file):
    conn = sqlite3.connect(db_file)
    return conn

def create_table(conn):
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS supertrend (
        symbol TEXT NOT NULL,
        Time TEXT NOT NULL,
        open REAL,
        high REAL,
        low REAL,
        close REAL,
        volume INTEGER,
        tr REAL,
        atr REAL,
        volume_factor REAL,
        upperband REAL,
        lowerband REAL,
        super_trend REAL,
        signal INTEGER,
        position INTEGER,
        buy_sell TEXT,
        is_uptrend INTEGER,
        PRIMARY KEY (symbol, Time)
    );
    """
    conn.execute(create_table_sql)
    conn.commit()

def fetch_data_from_db(conn, symbol):
    query = "SELECT * FROM supertrend WHERE symbol = ?"
    df = pd.read_sql_query(query, conn, params=(symbol,))
    return df

def insert_initial_data(conn, df, symbol):
    for index, row in df.iterrows():
        query = """
        INSERT INTO supertrend (symbol, Time, open, high, low, close, volume)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        ON CONFLICT(symbol, Time)
        DO UPDATE SET open=excluded.open, high=excluded.high, low=excluded.low, close=excluded.close, volume=excluded.volume;
        """
        conn.execute(query, (symbol, row['Time'].strftime('%Y-%m-%d %H:%M:%S'), row['open'], row['high'], row['low'], row['close'], row['volume']))
    conn.commit()

def insert_supertrend_data(conn, df, symbol):
    for index, row in df.iterrows():
        query = """
        INSERT INTO supertrend (symbol, Time, open, high, low, close, volume, tr, atr, volume_factor, upperband, lowerband, super_trend, signal, position, buy_sell, is_uptrend)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ON CONFLICT(symbol, Time)
        DO UPDATE SET open=excluded.open, high=excluded.high, low=excluded.low, close=excluded.close, volume=excluded.volume, tr=excluded.tr, atr=excluded.atr, volume_factor=excluded.volume_factor, upperband=excluded.upperband, lowerband=excluded.lowerband, super_trend=excluded.super_trend, signal=excluded.signal, position=excluded.position, buy_sell=excluded.buy_sell, is_uptrend=excluded.is_uptrend;
        """
        conn.execute(query, (symbol, row['Time'].strftime('%Y-%m-%d %H:%M:%S'), row['open'], row['high'], row['low'], row['close'], row['volume'], row['tr'], row['atr'], row['volume_factor'], row['upperband'], row['lowerband'], row['super_trend'], row['signal'], row['position'], row['buy_sell'], row['is_uptrend']))
    conn.commit()

def get_latest_date_in_db(conn, symbol):
    query = "SELECT MAX(Time) FROM supertrend WHERE symbol = ?"
    cursor = conn.execute(query, (symbol,))
    result = cursor.fetchone()
    if result and result[0]:
        return datetime.strptime(result[0], '%Y-%m-%d %H:%M:%S')
    else:
        return None

def fetch_data(symbol, interval='1d', start=None, end=None):
    suffixes = ['.NS', '.BO']
    for suffix in suffixes:
        try:
            data = yf.download(symbol + suffix, interval=interval, start=start, end=end, progress=False)
            if not data.empty:
                data.drop(columns='Adj Close', inplace=True)
                data.rename(columns={'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'}, inplace=True)
                data['Time'] = data.index
                data.reset_index(drop=True, inplace=True)
                return data
        except Exception as e:
            print(f"Failed to fetch data for {symbol+suffix}: {e}")
    return pd.DataFrame()

def DataFetcher(symbol, interval='1d'):
    conn = create_connection(DB_FILE)
    create_table(conn)
    latest_date = get_latest_date_in_db(conn, symbol)
    if latest_date:
        start_date = latest_date
        end_date = datetime.now()
        new_data = fetch_data(symbol, interval, start=start_date, end=end_date)
    else:
        new_data = fetch_data(symbol, interval)
    if not new_data.empty:
        insert_initial_data(conn, new_data, symbol)
    existing_data = fetch_data_from_db(conn, symbol)
    conn.close()
    return existing_data

def calculate_super_trend(df, multiplier, period):
    df = df.reset_index(drop=True)
    df['tr'] = np.maximum(df['high'] - df['low'],
                          np.abs(df['high'] - df['close'].shift(1)),
                          np.abs(df['low'] - df['close'].shift(1)))
    df['atr'] = df['tr'].rolling(window=period).mean()
    df['volume_factor'] = df['volume'] / df['volume'].rolling(window=period).mean()

    upperband = np.zeros(len(df))
    lowerband = np.zeros(len(df))
    super_trend = np.zeros(len(df))
    in_uptrend = [True] * len(df)

    for i in range(1, len(df)):
        current_close = df['close'].iloc[i]
        previous_close = df['close'].iloc[i - 1]
        previous_super_trend = super_trend[i - 1]

        if current_close > previous_super_trend:
            in_uptrend[i] = True
        else:
            in_uptrend[i] = False

        if in_uptrend[i]:
            lowerband[i] = ((df['high'].iloc[i] + df['low'].iloc[i]) / 2) - (multiplier * df['atr'].iloc[i] * df['volume_factor'].iloc[i])
            super_trend[i] = max(lowerband[i], previous_super_trend) if in_uptrend[i - 1] else lowerband[i]
        else:
            upperband[i] = ((df['high'].iloc[i] + df['low'].iloc[i]) / 2) + (multiplier * df['atr'].iloc[i] * df['volume_factor'].iloc[i])
            super_trend[i] = min(upperband[i], previous_super_trend) if not in_uptrend[i - 1] else upperband[i]

    df['super_trend'] = super_trend
    df['upperband'] = upperband
    df['lowerband'] = lowerband
    df['is_uptrend'] = in_uptrend
    df['signal'] = np.where(df['close'] > df['super_trend'], 1, -1)
    df['position'] = df['signal'].diff()
    df['buy_sell'] = df['position'].apply(lambda x: 'Buy' if x == 2 else 'Sell' if x == -2 else 'Hold')
    return df

def calculate_super_trend_for_new_data(df, period, multiplier, start_index):
    df = df.sort_values('Time').reset_index(drop=True)
    df.loc[start_index:, 'tr'] = np.maximum(df.loc[start_index:, 'high'] - df.loc[start_index:, 'low'],
                                            np.abs(df.loc[start_index:, 'high'] - df['close'].shift(1)),
                                            np.abs(df.loc[start_index:, 'low'] - df['close'].shift(1)))
    df.loc[start_index:, 'atr'] = df['tr'].rolling(window=period).mean()
    df.loc[start_index:, 'volume_factor'] = df['volume'] / df['volume'].rolling(window=period).mean()
    df['upperband'] = df['upperband'].fillna(0).astype(float)
    df['lowerband'] = df['lowerband'].fillna(0).astype(float)
    df['super_trend'] = df['super_trend'].astype(float)
    df['is_uptrend'] = df['is_uptrend'].fillna(False).astype(bool)

    for i in range(start_index, len(df)):
        current_close = df['close'].iloc[i]
        previous_close = df['close'].iloc[i - 1]
        previous_super_trend = df['super_trend'].iloc[i - 1]

        if current_close > previous_super_trend:
            df.at[i, 'is_uptrend'] = True
        else:
            df.at[i, 'is_uptrend'] = False

        if df.at[i, 'is_uptrend']:
            df.at[i, 'lowerband'] = ((df['high'].iloc[i] + df['low'].iloc[i]) / 2) - (multiplier * df['atr'].iloc[i] * df['volume_factor'].iloc[i])
            df.at[i, 'super_trend'] = max(df['lowerband'].iloc[i], previous_super_trend) if df['is_uptrend'].iloc[i - 1] else df['lowerband'].iloc[i]
        else:
            df.at[i, 'upperband'] = ((df['high'].iloc[i] + df['low'].iloc[i]) / 2) + (multiplier * df['atr'].iloc[i] * df['volume_factor'].iloc[i])
            df.at[i, 'super_trend'] = min(df['upperband'].iloc[i], previous_super_trend) if not df['is_uptrend'].iloc[i - 1] else df['upperband'].iloc[i]

    df.loc[start_index:, 'signal'] = np.where(df.loc[start_index:, 'close'] > df.loc[start_index:, 'super_trend'], 1, -1)
    df.loc[start_index:, 'position'] = df.loc[start_index-1:, 'signal'].diff()
    df.loc[start_index:, 'buy_sell'] = df.loc[start_index:, 'position'].apply(lambda x: 'Buy' if x == 2 else 'Sell' if x == -2 else 'Hold')
    return df

def calculate_sub(historical_df, new_df, period, multiplier):
    required_columns = ['symbol', 'Time', 'open', 'high', 'low', 'close', 'volume',
                        'tr', 'atr', 'volume_factor', 'upperband', 'lowerband',
                        'super_trend', 'signal', 'position', 'buy_sell', 'is_uptrend']

    new_df = new_df.copy()
    for col in required_columns:
        if col not in new_df.columns:
            new_df[col] = pd.NA

    new_df = new_df[required_columns]
    combined_df = pd.concat([historical_df, new_df], ignore_index=True)
    combined_df = combined_df.reset_index(drop=True)
    new_data_start_index = len(historical_df)
    combined_df = calculate_super_trend_for_new_data(combined_df, period, multiplier, new_data_start_index)
    new_data_result = combined_df.iloc[new_data_start_index:]
    return new_data_result

def check_and_insert_data(df, symbol, multiplier, period):
    conn = create_connection(DB_FILE)
    create_table(conn)
    cursor = conn.cursor()
    cursor.execute('SELECT MAX(Time) FROM supertrend WHERE symbol=?  AND super_trend IS NOT NULL', (symbol,))
    latest_time = cursor.fetchone()[0]

    if latest_time:
        latest_time = pd.to_datetime(latest_time)

    if not latest_time:
        df = calculate_super_trend(df, multiplier, period)
        insert_supertrend_data(conn, df, symbol)
        conn.close()
        return df
    else:
        cursor.execute('SELECT * FROM supertrend WHERE symbol=? ORDER BY Time DESC LIMIT ?', (symbol, period + 1))
        rows = cursor.fetchall()
        if rows:
            columns = [desc[0] for desc in cursor.description]
            historical_df = pd.DataFrame(rows, columns=columns)
            historical_df['Time'] = pd.to_datetime(historical_df['Time'])
            second_latest_time = historical_df['Time'].iloc[1]
            historical_df = historical_df[historical_df['Time'] <= second_latest_time]
            new_df = df[df['Time'] > second_latest_time]
            if not new_df.empty:
                combined_df = calculate_sub(historical_df, new_df, period, multiplier)
                insert_supertrend_data(conn, combined_df, symbol)
                conn.close()
                return combined_df
    conn.close()
    return df


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tqdm import tqdm
df= pd.read_csv('Paramns_stocks.csv')
all_stocks_data = []

for _, row in tqdm(df.iterrows(), desc="Fetched data of ", unit="stock"):
  try:
    df = DataFetcher(row['symbol'])
    df['Time'] = pd.to_datetime(df['Time'])
    result_df = check_and_insert_data(df, row['symbol'], multiplier=row['multiplier'], period=row['period'])
    temp_df = result_df[['symbol','Time', 'close', 'buy_sell']].tail(1)
    all_stocks_data.append(temp_df)
  except:
    continue

Fetched data of : 2121stock [14:25,  2.45stock/s]


In [ ]:
all_stocks_df = pd.concat(all_stocks_data)
all_stocks_df=all_stocks_df[all_stocks_df['buy_sell'] != 'Hold']
all_stocks_df.to_csv('BRUTE_SuperTrend_all_stocks.csv', index=False)

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
sender_email = "bhuvanesh.valiveti@gmail.com"
receiver_emails = ["bhuvanesh.valiveti@gmail.com","guruteja26@gmail.com","manas.baggu.official@gmail.com","rajamanohar931@gmail.com","manishsikakolli@gmail.com","tpvsssaketh@gmail.com"]
subject = "BRUTE Super Trend Stocks CSV File Attached"
body = "Please find attached the CSV file of BRUTE SuperTrend Stratergy."

# Create SMTP session
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(sender_email, "bcea clul fryl htqw")  # Replace "your_password" with your actual password

# Create message container
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = ", ".join(receiver_emails)
msg['Subject'] = subject

# Attach body to email
msg.attach(MIMEText(body, 'plain'))

# Attach CSV file to email
with open("BRUTE_SuperTrend_all_stocks.csv", "rb") as attachment:
    part = MIMEApplication(attachment.read(), Name="BRUTE_SuperTrend_all_stocks.csv")

part['Content-Disposition'] = f'attachment; filename="BRUTE_SuperTrend_all_stocks.csv"'
msg.attach(part)

# Send email
server.send_message(msg)

# Close SMTP session
server.quit()

(221,
 b'2.0.0 closing connection d2e1a72fcca58-705ccb3d268sm5495908b3a.101 - gsmtp')

In [ ]:
%%time
df = DataFetcher('RPOWER')
df['Time'] = pd.to_datetime(df['Time'])
result_df = check_and_insert_data(df, 'RPOWER', multiplier=3, period=14)
result_df.tail(1)

CPU times: user 95.2 ms, sys: 47.2 ms, total: 142 ms
Wall time: 234 ms


,symbol,Time,open,high,low,close,volume,tr,atr,volume_factor,upperband,lowerband,super_trend,signal,position,buy_sell,is_uptrend
14,RPOWER,2024-06-14,31.280001,32.41,30.299999,31.32,76469832,2.110001,1.447857,1.331018,0.0,25.573629,25.775003,1,0.0,Hold,True
